In [0]:
%pip install openpyxl

import pandas as pd
import numpy as np

## Bring in Cleaned Data

In [0]:
catalog = 'prd_mega'
schema = 'sdgreg25'
clean_name = 'ma_survey_clean'
labels_name = 'ma_survey_labels'
variables_name = 'ma_survey_variables'

# responses
rdf = spark.read.table(f'{catalog}.{schema}.{clean_name}')

# labels
ldf = spark.read.table(f'{catalog}.{schema}.{labels_name}')

# variables
vdf = spark.read.table(f'{catalog}.{schema}.{variables_name}')



In [0]:
# display(rdf)
# display(ldf)
# display(vdf.toPandas()['standalone_mis_name'])

## Scoring Funcs

In [0]:
def label_quartiles(df, score_col = 'total_score'):
    # Calculate quartiles
    q1, q2, q3 = df[score_col].quantile([.25, .50, .75])
    print('q1: ',q1)
    print('q2: ',q2)
    print('q3: ',q3)

    # Define function to label quartile
    def label_quartile(score):
        if score <= q1:
            return "Q1 (Lowest)"
        elif score <= q2:
            return "Q2"
        elif score <= q3:
            return "Q3"
        else:
            return "Q4 (Highest)"

    # Apply quartile label
    df["quartile"] = df[score_col].apply(label_quartile)

    return df


In [0]:
def score(row, questions, dicts:list =[]):
    caseid = row['caseid']
    answer_row = rdf.toPandas()[rdf.toPandas()['caseid'] == caseid].squeeze()
    
    score = 0
    total_questions = 0
    for q in questions:
        all_answers = answer_row[q]

        # questions for this section needs to fit into the answer_scores, text_impediments, or text_boosts
        q = q.replace('mis_financial_','').replace('mis_me_','').replace('mis_', '').replace('e_cohesion_', '')
        print(q)

        # weed out the questions people did not answer
        if not pd.isna(all_answers) and not all_answers == '':

            # make sure this is a question we are scoring
            for dic in dicts:
                if q in dic.keys():
                    answers = all_answers.split(' ')
                    for answer in answers:
                        total_questions += 1
                        if isinstance(dic[q],dict):
                            to_add = dic[q][answer]
                        else:
                            to_add = dic[q]
                        score += to_add
            
    return [score, total_questions]

In [0]:
def score_choices(row, col_ans_dict,score=0):
    for col in col_ans_dict.keys():
        answers = row[col]
        if not pd.isna(answers) and not answers == '':
            answers = answers.split(' ')
            for answer in answers:
                if not pd.isna(answer) and not answer == '':
                    score += col_ans_dict[col][answer]
    
    return score

def count_questions_answered(row, col_ans_dict,score=0):
    qas = 0
    for col in col_ans_dict.keys():
        answers = row[col]
        if not pd.isna(answers) and not answers == '':
            qas += 1
    
    return qas

def score_text(row,text_ans_dict,score=0):
    for col in text_ans_dict.keys():
        answer = row[col]
        if not pd.isna(answer) and not answer == '':
            score += text_ans_dict[col]

    return score

In [0]:
import pandas as pd
import plotly.express as px
import numpy as np

def plot_df(df, num=10):
    # -----------------------------
    # Chart 1: Horizontal Bar Chart of MAs by Score
    # -----------------------------
    df_sorted = df.sort_values("total_score", ascending=False)[:num]
    fig = px.bar(
        df_sorted,
        x="total_score",
        y="ma_name",
        orientation='h',
        title="Total Score by Managing Authority",
        category_orders={"ma_name": df_sorted.sort_values("total_score")['ma_name'].tolist()}
    )
    fig.update_layout(xaxis_title="Total Score", yaxis_title="Managing Authority")
    fig.show()

    # -----------------------------
    # Chart 2: Top 10 MAs by Score
    # -----------------------------
    display(df_sorted[['ma_name','ms_name','total_score']])

    # -----------------------------
    # Chart 3: Histogram with Percentile Lines
    # -----------------------------
    percentiles = np.percentile(df["total_score"], [25, 50, 75])
    fig = px.histogram(df, x="total_score", nbins=10, title="Distribution of Scores with Percentiles")
    for p, label in zip(percentiles, ["25th", "50th", "75th"]):
        fig.add_vline(x=p, line_dash="dash", line_color="red", annotation_text=f"{label}: {p:.1f}", annotation_position="top right")
    fig.update_layout(xaxis_title="Total Score", yaxis_title="Frequency")
    fig.show()

    # -----------------------------
    # Chart 4: Average Score by Region (Member State)
    # -----------------------------
    score_by_country = df.groupby("ms_name", as_index=False)["total_score"].mean()
    fig = px.choropleth(
        score_by_country,
        locations="ms_name",
        locationmode="country names",
        color="total_score",
        color_continuous_scale="Viridis",
        title="Average Total Score by Country"
    )
    fig.show()

    # -----------------------------
    # Chart 5: Count MAs per country
    # -----------------------------
    ma_counts = df.groupby("ms_name")["ma_name"].nunique().reset_index()
    ma_counts.columns = ["Member State", "Number of MAs"]
    ma_counts = ma_counts.sort_values("Number of MAs", ascending=False)
    fig = px.bar(
        ma_counts,
        x="Member State",
        y="Number of MAs",
        title="Number of Managing Authorities per Member State",
        text="Number of MAs"
    )
    fig.update_traces(textposition="outside")
    fig.update_layout(yaxis_title="Number of MAs", xaxis_title="Member State")
    fig.show()

    # -----------------------------
    # Chart 6: Country vs Quartile Count (Grouped Stacked Bar)
    # -----------------------------
    df = label_quartiles(df)
    counts = df.groupby(["ms_name", "quartile"]).size().reset_index(name="count")

    fig = px.bar(
        counts,
        x="ms_name",
        y="count",
        color="quartile",
        title="Number of Managing Authorities per Country by Quartile",
        barmode="stack"
    )
    fig.update_layout(xaxis_title="Member State", yaxis_title="Number of MAs")
    fig.show()

    # -----------------------------
    # Chart 7: Managing Authorities by Score and Quartile (Sorted Bar Chart)
    # -----------------------------
    df_sorted = df.sort_values("total_score", ascending=False)

    fig = px.bar(
        df_sorted,
        x="total_score",
        y="ma_name",
        color="quartile",
        orientation="h",
        title="Managing Authorities by Score and Quartile",
        category_orders={"ma_name": df_sorted["ma_name"].tolist()}
    )
    fig.update_layout(xaxis_title="Total Score", yaxis_title="Managing Authority")
    fig.show()

    # -----------------------------
    # Chart 8: Strip Plot for Visual Density by Country
    # -----------------------------
    fig = px.strip(
        df,
        x="ms_name",
        y="total_score",
        color="quartile",
        title="Distribution of MA Scores by Country and Quartile",
        hover_data=["ma_name"]
    )
    fig.update_layout(yaxis_title="Total Score", xaxis_title="Member State")
    fig.show()

    

## Technological Foresight and Engagement


In [0]:
clean_scoring = {
    'prior_api_considerations':{
        'financial_information':1, 
        'output_and_result_indicators':1, 
        'financial_instrument_information':1, 
        'no':0
        },
    'reason_for_no_api_use':{
        'unaware':-1, 
        'under_development':1, # proactive measure in favor of
        'lack_of_information':-1,
        'no_integration_developed':-1, 
        'managed_by_higher_level':0, # not in their court
        'lack_of_systems_integration':-1, 
        'diffs_in_data_formats':-1, 
        'budgetary_restrictions':-1, 
        'manual_deduplication':-1, 
        'data_quality_checks':-1, 
        'likes_as_is':-1, 
        'too_difficult':-1, 
        'other':-1
        },
    'resources_to_implement_api':{
        'external_buy_in':1, 
        'no_interest':0, 
        'field_compatibility':1, 
        'no_support_needed':0, 
        'unaware':0, 
        'budget':1, 
        'training':1, 
        'regulatory_changes':1, 
        'technical_assistance':1, 
        'improved_documentation':1, 
        'peer_experience':1, 
        'other':1
        },
    'reporting_clarity':{
        'yes':2, 
        'somewhat':1, 
        'no':0, 
        'does_not_know':0
        },
    'arachne_use':{
        'yes':1, 
        'no':0, 
        'unsure':0
        },  
   }

clean_texts = {
    'regional_national_integration_wants':1, 
    'regulatory_improvement_suggestions':1, 
    'missed_automation_opportunities':1, 
    'reporting_integration_challenges':1
    }

In [0]:
pd_rdf = rdf.toPandas()
mas_clean = pd_rdf.fillna('').loc[:,['ma_name','ms_name','caseid']]

mas_clean['clean_score'] = pd_rdf.apply(lambda row: score_choices(row,clean_scoring), axis=1)
mas_clean['clean_text_score'] = pd_rdf.apply(lambda row: score_text(row,clean_texts), axis=1)
mas_clean['total_score'] = mas_clean['clean_text_score'] + mas_clean['clean_score']

plot_df(mas_clean)

## Technology Sophistication of Managing Authorities

In [0]:
tsma_systems = {
        'single_or_separate_system':{
        'results_integrated_e_cohesion':1, 
        'financial_integrated_e_cohesion':1, 
        'one_mis_separate_e_cohesion':1, 
        'all_integrated': 5, 
        'all_separate':0
        },
    }

tsma_scores = {

    'system_integration_level':{
        'fully':3, 
        'partially':2, 
        'barely':1, 
        'not_at_all':0, 
        'na':0
        },
    }

tsma_boiler_scores = {
    'meeting_needs': {'fully':2,
        'partially':1,
        'minimally':0,
    },
    'updates':{
        'yes_maintenance':2,
        'yes_maintenance_and_improvements':3,
        'no_but_adds_regulation':1,
        'not_at_all':0
    },
    'api_data_transmission':{
        'yes':2,
        'no':0,
        'no_but_pursuing':1,
        'does_not_know':0,
    },
    'barriers_to_api_implementation':{
        'lack_of_technical_capacity':-1,
        'budget_constraints':-1,
        'regulatory_restrictions':-1,
        'legacy_issues':-1,
        'security_concerns':-1,
        'satisfied':-1,
        'other':-1,
    },
 }

tsma_boiler_texts = {
            'explain_partially':0,
            'other_api_barriers':1
            }

tsma_texts = {
    'system_integration_level_partial_barely':0
    }

def apply_hdrs(boiler_dict,parent_dict):
    tsma_boiler_headers = ['mis_','mis_financial_','mis_me_','e_cohesion_']
    for header in tsma_boiler_headers:
        for q in boiler_dict.keys():
            parent_dict[header+q] = boiler_dict[q]
    
    return parent_dict

tsma_scores = apply_hdrs(tsma_boiler_scores,tsma_scores)
tsma_texts = apply_hdrs(tsma_boiler_texts,tsma_texts)

In [0]:
pd_rdf = rdf.toPandas()
mas_tsma = pd_rdf.fillna('').loc[:,['ma_name','ms_name','caseid']]

mas_tsma['tsma_system_score'] = pd_rdf.apply(lambda row: score_choices(row,tsma_systems), axis=1)
mas_tsma['tsma_score'] = pd_rdf.apply(lambda row: score_choices(row,tsma_scores), axis=1)
mas_tsma['tsma_text_score'] = pd_rdf.apply(lambda row: score_text(row,tsma_texts), axis=1)
mas_tsma['tsma_count'] = pd_rdf.apply(lambda row: count_questions_answered(row,tsma_scores), axis=1)

# important to average by number of questions answered, not total score
mas_tsma['total_score'] = mas_tsma['tsma_system_score'] + (mas_tsma['tsma_text_score'] + mas_tsma['tsma_score'])/mas_tsma['tsma_count']

plot_df(mas_tsma)

## MA Data Processing Capability

In [0]:
mdpc_scores = {
    'sfc_submission_types':{
        'manual':0, 
        'api':2, 
        'excel':1, 
        'other':0,
        },
    'no_excel_upload':{
        'api':1,
        'mis_does_not_generate':0, 
        'unaware':0, 
        'error':0,
        },
    'data_validation_procedure':{'training':0, 
        'audits':0, 
        'double_entry':0, 
        'manual_check':0, 
        'other':0,
        },
    'operations_publishing_pipeline':{
        'fully_automatic':3, 
        'mostly_automatic':2, 
        'mostly_manual':1, 
        'fully_manual':0,
        },
     
    'operations_publishing_manual_steps':{
        'manual_creation':0, 
        'it_system_download':0, 
        'manual_publishing':0, 
        'api_planned':0, 
        'external_department_involved':0, 
        'manual_check':0, 
        'publication_not_working':0,
        },
    
    'arachne_submission':{
        'fully_automatic':3, 
        'mostly_automatic':2, 
        'mostly_manual':1, 
        'fully_manual':0,
        }
}

mdpc_texts = {'manual_steps_explanation':0}

In [0]:
pd_rdf = rdf.toPandas()
mas_mdpc = pd_rdf.fillna('').loc[:,['ma_name','ms_name','caseid']]

mas_mdpc['mdpc_score'] = pd_rdf.apply(lambda row: score_choices(row,mdpc_scores), axis=1)
mas_mdpc['mdpc_text_score'] = pd_rdf.apply(lambda row: score_text(row,mdpc_texts), axis=1)
mas_mdpc['total_score'] = mas_mdpc['mdpc_score'] + mas_mdpc['mdpc_text_score']

plot_df(mas_mdpc)

## Cross-System Integration Within Country

In [0]:
csi_scores = {
    'regional_national_integration_existing':{
        'population_registry':1, 
        'other_mis':1, 
        'public_procurement_system':1, 
        'e_government':1, 
        'keep':1, 
        'national_data_bank':1, 
        'funding_management':1, 
        'index':1, 
        'specific_national_registry':1, 
        'financial_management_system':1,
        },
    'eu_integration_existing':{
        'public_procurement_system':0, 
        'other_mis':0, 
        'arachne':0, 
        'keep':0, 
        'sfc':0,
        'index':0
        },
    'eu_integration_wants':{
        'ims':0, 
        'public_procurement_system':0,
        'other_mis':0, 
        'arachne':0, 
        'keep':0, 
        'funding_management':0, 
        'sfc':0, 
        'index':0, 
        'specific_national_registry':0
        },
    'table_1_to_2_data_origin':{
        'mis':0, 
        'e_cohesion':0, 
        'other':0,
        },
    'table_5_to_10_data_origin':{
        'mis':0, 
        'e_cohesion':0, 
        'other':0,
        },
    'table_12_data_origin':{
        'mis':0, 
        'e_cohesion':0, 
        'other':0,
        },
    'sfc_automation_level':{
        'fully':0, 
        'partially':0, 
        'not_at_all':0, 
        'other':0,
        },
    'operation_data_origin':{
        'me_mis':0,
        'financial_mis':0, 
        'e_cohesion':0, 
        'other':0,
        },
    'arachne_information_origin':{
        'me_mis':0, 
        'financial_mis':0, 
        'e_cohesion':0, 
        'other':0
        }
}

csi_texts = {'operation_data_origin_other':0, 'other_arachne_information_origin':0}

In [0]:
pd_rdf = rdf.toPandas()
mas_csi = pd_rdf.fillna('').loc[:,['ma_name','ms_name','caseid']]

mas_csi['csi_score'] = pd_rdf.apply(lambda row: score_choices(row,csi_scores), axis=1)
mas_csi['csi_text_score'] = pd_rdf.apply(lambda row: score_text(row,csi_texts), axis=1)
mas_csi['total_score'] = mas_csi['csi_score'] + mas_csi['csi_text_score']

plot_df(mas_csi)

## Total Score

In [0]:
mas_mdpc_short = mas_mdpc.rename(columns={'total_score': 'score_mdpc'})
mas_csi_short = mas_csi.rename(columns={'total_score': 'score_csi'})
mas_clean_short = mas_clean.rename(columns={'total_score': 'score_clean'})
mas_tsma_short = mas_tsma.rename(columns={'total_score': 'score_tsma'})

df_merged = mas_mdpc_short[['caseid', 'ma_name', 'ms_name', 'score_mdpc']]\
    .merge(mas_csi_short[['caseid', 'score_csi']], on='caseid', how='inner')\
    .merge(mas_clean_short[['caseid', 'score_clean']], on='caseid', how='inner')\
    .merge(mas_tsma_short[['caseid', 'score_tsma']], on='caseid', how='inner')

# Weights Taken from Transformation Readiness Pre-Analysis Doc
df_merged['total_score'] = (
    0.3 * df_merged['score_mdpc'] +
    0.2 * df_merged['score_csi'] +
    0.4 * df_merged['score_clean'] +
    0.1 * df_merged['score_tsma']
)

total_df = df_merged[['caseid', 'ma_name', 'ms_name', 'score_mdpc', 'score_csi', 'score_clean', 'score_tsma', 'total_score']]

plot_df(total_df, 40)

In [0]:
attendee_path = r'/Volumes/prd_mega/sdgreg25/vdgreg25/Documents/Workshop/managing-authorities-emails-and-urls.xlsx'
attendees = pd.read_excel(attendee_path, sheet_name='Workshop participants full list', engine='openpyxl')

In [0]:
ws_case_ids = attendees['Caseid'].unique()
# display(len(case_ids))
ws_df = total_df[total_df['caseid'].isin(ws_case_ids)]
ws_df = label_quartiles(ws_df)
# display(ws_df)

score_clean = 'Foresight and Engagement'
score_tsma = 'Sophistication'
score_csi = 'Cross-System Integration Within Country'
score_mdpc = 'Data Processing Capability'
ws_df = ws_df.rename(columns={'score_clean':score_clean,'score_tsma':score_tsma,'score_csi':score_csi,'score_mdpc':score_mdpc})
plot_df(ws_df,num=20)
# for i,row in ws_df[ws_df['ma_name']=='Ministry of Investments and European Projects'].iterrows():
#     print(row['caseid'])

In [0]:
#  Compare All MAs Across Dimensions

melted = ws_df.melt(id_vars=["ma_name"], value_vars=[score_mdpc, score_csi, score_tsma, score_clean],
                 var_name="Dimension", value_name="Score")

fig = px.bar(
    melted,
    x="ma_name",
    y="Score",
    color="Dimension",
    barmode="group",
    title="Maturity Scores by Dimension per Managing Authority"
)
fig.update_layout(xaxis_title="Managing Authority", yaxis_title="Score")
fig.show()

In [0]:
fig = px.box(melted, x="Dimension", y="Score", points="all", title="Distribution of Scores per Maturity Dimension")
fig.update_layout(yaxis_title="Score")
fig.show()


In [0]:
categories = [score_mdpc, score_csi, score_tsma, score_clean]

# Average scores per dimension by country
avg_by_country = ws_df.groupby("ms_name")[categories].mean().reset_index()

melted_country = avg_by_country.melt(
    id_vars="ms_name",
    value_vars=categories,
    var_name="Dimension",
    value_name="Average Score"
)

import plotly.express as px

fig = px.bar(
    melted_country,
    x="ms_name",
    y="Average Score",
    color="Dimension",
    barmode="group",
    title="Average Maturity Scores by Dimension per Country"
)
fig.update_layout(xaxis_title="Member State", yaxis_title="Average Score")
fig.show()

fig = px.line_polar(
    melted_country,
    r="Average Score",
    theta="Dimension",
    color="ms_name",
    line_close=True,
    title="Technological Maturity Profiles per Country"
)
fig.update_traces(fill="toself")
fig.show()


In [0]:
# Calculate Ability to Execute
ws_df["ability_to_execute"] = ws_df[[score_tsma, score_mdpc, score_csi]].mean(axis=1)

# X = Vision (A), Y = Ability (B, C, D)
x = score_clean  # Technological Foresight and Engagement
y = 'ability_to_execute'

fig = px.scatter(
    ws_df,
    x=x,
    y=y,
    # text="ma_name",
    color="ms_name",  # Optional: color by country
    title="Maturity of Managing Authorities",
    labels={
        x: "High Completeness of Vision (A)",
        y: "High Ability to Execute (B, C, D)"
    },
    hover_data=["ma_name", "ms_name", x, y]
)

# Add quadrant lines at medians (or percentiles)
x_median = ws_df[x].median()
y_median = ws_df[y].median()

fig.add_vline(x=x_median, line_dash="dash", line_color="gray")
fig.add_hline(y=y_median, line_dash="dash", line_color="gray")

# Optional: quadrant labels
fig.add_annotation(x=x_median + 2, y=y_median + .2, text="Leaders", showarrow=False)
fig.add_annotation(x=x_median - 2, y=y_median + .2, text="Challengers", showarrow=False)
fig.add_annotation(x=x_median + 2, y=y_median - .2, text="Visionaries", showarrow=False)
fig.add_annotation(x=x_median - 2, y=y_median - .2, text="Niche Players", showarrow=False)

fig.update_traces(textposition="top center")
fig.update_layout(xaxis=dict(title="Completeness of Vision (A)"),
                  yaxis=dict(title="Ability to Execute (B, C, D)"),
                  height=450,
                  width=600)

fig.show()
